<a href="https://colab.research.google.com/github/brownbobcat/animal-backend-service/blob/main/LoRa_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title ⛽️ Check GPU availability
!nvidia-smi
import torch, platform
print(f'Torch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    capability = torch.cuda.get_device_capability(0)
    print(f'GPU: {device_name} (capability {capability[0]}.{capability[1]})')
print(f'Python: {platform.python_version()}')

Wed Dec 10 06:29:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   44C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
!pip install -q --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.2 MB/s eta 0:00:00


In [2]:
!pip install -q transformers accelerate datasets peft sentence-transformers einops


In [3]:
#@title 📁 Upload and extract your zip file
import shutil, zipfile, os
from pathlib import Path
from google.colab import files
project_dir = Path('/content/project')
if project_dir.exists():
    shutil.rmtree(project_dir)
project_dir.mkdir(parents=True, exist_ok=True)
print('Select your project zip (e.g., silvaco_training_data.zip)...')
uploaded = files.upload()
for filename, data in uploaded.items():
    local_path = Path(filename)
    with open(local_path, 'wb') as f:
        f.write(data)
    if local_path.suffix == '.zip':
        print(f'Extracting {local_path} ...')
        with zipfile.ZipFile(local_path, 'r') as zip_ref:
            zip_ref.extractall(project_dir)
print('Extraction complete. Listing top-level contents:')
for path in project_dir.iterdir():
    print(' -', path)

Select your project zip (e.g., silvaco_training_data.zip)...


Saving silvaco_training_data.zip to silvaco_training_data.zip
Extracting silvaco_training_data.zip ...
Extraction complete. Listing top-level contents:
 - /content/project/data
 - /content/project/src


In [4]:
DATASET_PATH = "/content/silvaco_dataset_train.json"

In [32]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": DATASET_PATH})

train_dataset = dataset["train"]
eval_dataset = train_dataset.train_test_split(test_size=0.05, seed=42)["test"]

print("Train samples:", len(train_dataset))
print("Validation samples:", len(eval_dataset))
print("Columns:", train_dataset.column_names)

Train samples: 713
Validation samples: 36
Columns: ['instruction', 'input', 'output']


In [34]:
def format_row(row):
    instruction = row["instruction"]
    model_input = row.get("input", "")
    output = row["output"]

    if model_input:
        row["text"] = f"Instruction: {instruction}\nInput: {model_input}\nResponse: {output}"
    else:
        row["text"] = f"Instruction: {instruction}\nResponse: {output}"
    return row

train_dataset = train_dataset.map(format_row)
eval_dataset = eval_dataset.map(format_row)

Map:   0%|          | 0/713 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "Qwen/Qwen2-0.5B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True
)


In [36]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,081,344 || all params: 495,114,112 || trainable%: 0.2184


In [37]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": DATASET_PATH})

train_dataset = dataset["train"]
eval_dataset  = train_dataset.train_test_split(test_size=0.05, seed=42)["test"]

print(train_dataset.column_names)
print(train_dataset[0])




['instruction', 'input', 'output']
{'instruction': "I'm designing a voltage-controlled oscillator for a 2.5GHz RF application requiring comprehensive phase noise characterization. The topology employs a cross-coupled NMOS pair (XM1, XM2) with symmetric LC tank loading, where each tank consists of spiral inductors (XL1, XL2) and MOS varactors (XC1, XC2) for frequency tuning. The design operates with a 2V supply (V1) and utilizes a 1mA bias current (I2) from a current mirror configuration (M1, M2) with 100:1 scaling ratio.\n\nKey parameters include: spiral inductors with 60μm radius, 10μm trace width, 2.5 turns, and 2μm spacing; varactors with 40μm length and 0.58μm width in 5-finger, 2-multiplier configuration; cross-coupled transistors with 350nm effective length, 5μm width, and 50-finger layout.\n\nFor phase noise analysis, configure harmonic balance using shooting method with 10ns stabilization time (tstab=10n) and 0.1 error tolerance (ese=0.1). Perform noise simulation from 1kHz to 

In [38]:
dataset = load_dataset("json", data_files={"train": DATASET_PATH})

train_dataset = dataset["train"]
eval_dataset  = train_dataset.train_test_split(test_size=0.05, seed=42)["test"]

print(train_dataset.column_names)


['instruction', 'input', 'output']


In [39]:
def format_example(example):
    # If input is empty, skip it cleanly
    inp = example["input"] if example["input"] else ""

    example["text"] = (
        f"Instruction:\n{example['instruction']}\n\n"
        f"Input:\n{inp}\n\n"
        f"Output:\n{example['output']}"
    )
    return example

train_dataset = train_dataset.map(format_example)
eval_dataset  = eval_dataset.map(format_example)


In [40]:
train_dataset[0]["text"][:400]


"Instruction:\nI'm designing a voltage-controlled oscillator for a 2.5GHz RF application requiring comprehensive phase noise characterization. The topology employs a cross-coupled NMOS pair (XM1, XM2) with symmetric LC tank loading, where each tank consists of spiral inductors (XL1, XL2) and MOS varactors (XC1, XC2) for frequency tuning. The design operates with a 2V supply (V1) and utilizes a 1mA b"

In [41]:
from functools import partial

def tokenize(example, tokenizer):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    # causal LM training target
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenize_fn = partial(tokenize, tokenizer=tokenizer)

train_dataset = train_dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=train_dataset.column_names  # removes instruction/input/output/text
)

eval_dataset = eval_dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=eval_dataset.column_names
)




Map:   0%|          | 0/713 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [42]:
print(train_dataset.column_names)


['input_ids', 'attention_mask', 'labels']


In [43]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,          # causal LM
)


In [44]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./qwen-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=20,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    report_to="none",
    label_names=["labels"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)



/tmp/ipython-input-714214334.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [45]:
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Epoch,Training Loss,Validation Loss
1,2.316700,2.205470
2,2.035000,1.995243
3,1.923500,1.939011


TrainOutput(global_step=270, training_loss=2.191389776159216, metrics={'train_runtime': 375.57, 'train_samples_per_second': 5.695, 'train_steps_per_second': 0.719, 'total_flos': 2358856877211648.0, 'train_loss': 2.191389776159216, 'epoch': 3.0})

In [46]:
trainer.save_model("./qwen_lora_silvaco")

In [47]:
tokenizer.save_pretrained("./qwen_lora_silvaco")

('./qwen_lora_silvaco/tokenizer_config.json',
 './qwen_lora_silvaco/special_tokens_map.json',
 './qwen_lora_silvaco/chat_template.jinja',
 './qwen_lora_silvaco/vocab.json',
 './qwen_lora_silvaco/merges.txt',
 './qwen_lora_silvaco/added_tokens.json',
 './qwen_lora_silvaco/tokenizer.json')